# Выполнил Евгений Гузеев

In [1]:
# Загрузим данные
# https://drive.google.com/file/d/1LBEZdnInnjnjqLvUAbG-QqiVd2J-unQy/view?usp=share_link
! gdown --id 1LBEZdnInnjnjqLvUAbG-QqiVd2J-unQy

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1LBEZdnInnjnjqLvUAbG-QqiVd2J-unQy
To: /content/fbert.csv
100% 2.16M/2.16M [00:00<00:00, 203MB/s]


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
#from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
#from pytorch_pretrained_bert import BertTokenizer, BertConfig
#from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [ ]:
#device = torch.device('cpu')

In [5]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 123.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.6 MB/s eta 0:00:00


In [6]:
from transformers import BertTokenizer

## Загрузка Датасета

In [7]:
df = pd.read_csv('fbert.csv')
# Заменим классы на 0, 1, 2, 3
df['target'] = df['target'].replace([1, 2, 3, 4], [0, 1, 2, 3])
df.head(5)

,processed,target
0,присылать анекдот предложить новость группа ре...,0
1,познавательный мем широкий охват уникальный ид...,0
2,хабра q a ответ любой тема it участник сообщес...,3
3,официальный канал сервис рбк инвестиция заботи...,2
4,помогать новичок веб разработчик опытный прогр...,3


In [8]:
# Create sentence and label lists
sentences = df.processed.values
labels = df.target.values
# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
#sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]


## Inputs

Next, import the BERT tokenizer, used to convert our text into tokens that correspond to BERT's vocabulary.

In [9]:
tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny2', do_lower_case=True)
#rubert_based_cased_sentence/
#cointegrated/rubert-tiny2


Применим  токенизатор к одному предложению и посмотрим что получится

In [ ]:
print(f'Оригинальный текст \n\n{sentences[0]}')

# Выведем токенизированное предложение
print(f'Токенизированный текст \n\n{tokenizer.tokenize(sentences[0])}')

# Выведем id токенов
print(f'Токен id \n\n {tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0]))}')

Оригинальный текст 

присылать анекдот предложить новость группа результат коллективный творчество участник практически анекдот подписчик вкус разный известно спорить лицо тонкий возвышенный вкус некоторый анекдот понравиться баян пояс проч пусть считать надоесть реклама хотеть шикарный анекдот слать шикарный группа включн казнный фильтр нецензурный выражение писать сообщение сообщество вопрос повод пост направлять автор всякий род нарушение порядок комментарий спам проч жаловаться право верхний угол каждый пост крестик юмор азохнвеять бесплатный щикарный анекдот
Токенизированный текст 

['присы', '##лать', 'анекдот', 'предложить', 'новость', 'группа', 'результат', 'коллектив', '##ны', '##и', 'творчество', 'участник', 'практически', 'анекдот', 'подпис', '##чик', 'вкус', 'раз', '##ны', '##и', 'известно', 'спорить', 'лицо', 'тонки', '##и', 'возвыш', '##енны', '##и', 'вкус', 'некото', '##ры', '##и', 'анекдот', 'понравиться', 'ба', '##ян', 'пояс', 'проч', 'пусть', 'считать', 'надое', '##ст

### Required Formatting

1. Добавить специальные токены начала и конца предложения.
2. Дополнить и усечь предложения одной длины.
3. В явном виде отделить реальные токены от padding-токенов с помощью `attention-mask`

#### Специальные токены

`[SEP]`
В конце каждого предложения необходимо добавить специальный токен `[SEP]`

`[CLS]`
Для задач классиифкации в начале каждого предложения необходимо добавить токен `[CLS]`


### Длина предложения и Attention Mask

Тексты в датасете имеют разную длину.

BERT имеет ограничения на входные предложения:

    1. Все предложения должны иметь одинаковую длину.
    2. Максимальная допустимая длина предложений 512 токенов

Для паддинга используется специальный токен  `[PAD]` с индексом 0 в словаре BERT'a

`[PAD]` токен добавляется для того чтобы сделать все предложения одной длины

### Токенизация данных

В библиотеке `transformers` есть функция `encode`, с помощью, которой удобно токенизировать предложения.

Но перед тем, как закодировать текст, найдем максимальную длину предложений.

In [ ]:
max_len = 0

# Циклом пробегаемся по каждому тексту
for sent in sentences:
    # Добавляем специальные токены [CLS] и [SEP]
    input_ids = tokenizer.encode(sent, add_special_tokens = True)
    # Обновляем максимальную длину предложения
    max_len = max(max_len, len(input_ids))

print(f'Максимальная длина предложения составляет:   {max_len}')

Максимальная длина предложения составляет:   980


### Усечем все предложения до длины 250

Все готово для токенизации.

Применим функцию `tokenizer.encode_plus` , которая выполнит необходимые шаги:
   1. Разобьет предложения на токены.
   2. Добавит специальные токены [CLS] и [SEP]
   3. Заменит токены на их ID
   4. Приведет все предложения к одной длине - max_len
   5. Создаст atention_mask

In [ ]:
# Разобьем все на токены
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                    sent,
                    add_special_tokens=True,            # Добавляем токены
                    max_length = 512,                   # Ограничиваем длину
                     pad_to_max_length = True,          # Заменяем пустоты [PAD]
                    return_attention_mask = True,       # Создаем маску
                    return_tensors = 'pt'               # Возвращаем  тензоры pytorch
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Конвертируем список тензоров
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2346: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-26-97b389fc9efe>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [ ]:
input_ids

tensor([[    2, 44871, 35104,  ...,     0,     0,     0],
        [    2, 48796,  1385,  ...,     0,     0,     0],
        [    2, 60090,  1232,  ...,     0,     0,     0],
        ...,
        [    2, 62777, 59661,  ...,     3,     0,     0],
        [    2, 12458, 43093,  ..., 38408,   705,     3],
        [    2, 53570,  3126,  ...,     0,     0,     0]])

In [ ]:
# Разбиваем на train и валидационный датасет
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.7 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print(f'train содержит {train_size} записей')
print(f'test содержит {val_size} записей')

train содержит 3055 записей
test содержит 1310 записей


#### Также создадим итератор с использованием класса `DataLoader` из библиотеки torch

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# Даталоадеру необходимо знать количество батчей
# Для текстовых задач рекомендуется небольшре количество батчей
batch_size = 64

# Создаем dataloader из train и val
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset), # при обучении батчи выбираются случайным образом
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = RandomSampler(val_dataset),
            batch_size = batch_size
        )

## Обучение модели

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertForMultipleChoice, BertForPreTraining, BertModel
model = BertForSequenceClassification.from_pretrained(
       'cointegrated/rubert-tiny2',
        num_labels = 4,
        output_attentions = False,
        output_hidden_states = False,
        )
model.to(device)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elemen

# Optimizer & Learning Rate scheduler

Почти все готово для обучения, осталось задать optimizer и scheduler, которые помогут нам с файнтьюном

Наиболее популярные конфигурации параметров:
* **Batch size: 16, 32**
* **Learning rate (Adam): 5e-5, 3e-5, 2e-5**
* **Number epochs: 2, 3, 4**

Параметр **eps** позволяет избежать деления на ноль

In [ ]:
optimizer = AdamW(model.parameters(),
                 lr = 2e-5,
                 eps = 1e-8)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Задаем количество эпох
epochs = 10

total_steps = len(train_dataloader) * epochs


learning_rate = get_linear_schedule_with_warmup(optimizer,
                                      num_warmup_steps = 0,
                                      num_training_steps = total_steps)

# Создаем планировщик, который после определенного числа шагов понижает learning_rate
scheduler = get_linear_schedule_with_warmup(optimizer,
                                           num_warmup_steps = 0,
                                           num_training_steps = total_steps)

## Петля обучения

Обучающая петля это один проход по данным

Ниже приведена петля обучения (training loop). На каждом проходе по данным мы проходим по всему обучающему и валидационному датасетам.

**Training:**

* Распаковать обучающие данные и метки
* Загрузить данные на GPU для ускорения
* Занулить градиенты с предыдущего шага
* Forward pass (подать данные на вход в нейросеть и пробросить их вперед)
* Backward pass (backpropogation - посчитать градиенты по всем параметрам с помощью обратного распространения ошибки)
* Обновить параметры с помощью optimizer.step()
* Посчитать статистики чтобы следить за обучением

**Evalution:**

* Распаковать валидационные данные и метки
* Загрузить данные на GPU для ускорения
* Forward pass (подать данные на вход в нейросеть и пробросить их вперед)
* Посчитать loss и статистики на валидационных данных, чтобы следить за обучением

In [ ]:
# Напишем функцию вычисления f1_score


def flat_f1(preds, labels):
    from sklearn.metrics import f1_score
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    metrics_f1 = f1_score(pred_flat, labels_flat, average = 'weighted')
    return metrics_f1


In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

Вспомогательная функция для отслеживания затраченного времени

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''Берем время в секундах и возвращаем в формате часы:минуты:секунды'''
    elapsed_rounded = int(round((elapsed)))

    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import random

# Замораживаем счетчик
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Создаем список для сохранения f1 и времени
training_stats = []
total_t0 = time.time()

# Для каждой эпохи...
for epoch_i in range(0, epochs):
    print('')
    print(f'======= Epoch {epoch_i + 1} / {epochs} ========')

    # Смотрим сколько идет обучение по train
    t0 = time.time()
    # Сбрасываем все потери для эпохи
    total_train_loss = 0
    # Ставим модель в режим train это такой режим, который может обновлять свои веса
    model.train()

    # Для каждого батча тренировочных данных
    for step, batch in enumerate(train_dataloader):

        # Прогресс обновляется каждые 40 батчей
        if step % 40 == 0 and not step == 0:
            # считаем время
            elapsed = format_time(time.time() - t0)

            # Прогресс отчета
            print(f'Batch {step} of {len(train_dataloader)},{elapsed}')

        # Распаковываем данные и отправляем на GPU
        #
        # Достаем из батчей id маски и лэйблы
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)


        # Зануляем градиенты модели с предыдущего шага
        model.zero_grad()

        # Делаем forward_pass
        res = model(b_input_ids,
                   token_type_ids = None,
                   attention_mask = b_input_mask,
                   labels = b_labels)

        # Получаем loss
        loss = res['loss']
        # Получаем вероятности классов
        logits = res['logits']

        # Добавляем наш loss в обучающий loss
        total_train_loss += loss.item()

        # Делаем backward pass чтобы посчитать градиенты
        loss.backward()

        # Нормы всех градиентов обрезаем до 1 чтобы предотвратить проблему взрывающихся градиентов
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Обновляем веса против градиента
        optimizer.step()

        # Обновляем learning_rate
        scheduler.step()
    # Считаем средний лосс
    avg_train_loss = total_train_loss / len(train_dataloader)
    # Считаем время как долго берет эпоха
    training_time = format_time(time.time() - t0)

    print()
    print(f'Средние потери на train {avg_train_loss}')
    print(f'Время эпохи {training_time}')

    print('')
    print('Сейчас идет валидациия...')

    t0 = time.time()

    # В валидационной части мы не обновляем веса модели поэтому ставим модель в режим eval
    model.eval()

    # Задаем переменные для лосса
    total_eval_f1 = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        # Достаем из батчей id маски и лэйблы
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Не считаем градиенты
        with torch.no_grad():
            res = model(b_input_ids,
                   token_type_ids = None,
                   attention_mask = b_input_mask,
                   labels = b_labels)


            # Получаем loss
            loss = res['loss']
            # Получаем вероятности классов
            logits = res['logits']

        # Считаем потери на валидации
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        labels_ids = b_labels.to('cpu').numpy()

        # Считаем f1
        total_eval_f1 += flat_f1(logits, labels_ids)

    # Отчет о финальной метрике
    avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    print(f'f1_score--->  {avg_val_f1}')

    # Считаем потери на всех батчах
    avg_val_loss = total_eval_loss / len(validation_dataloader)

    # Считаем время
    validation_time = format_time(time.time()- t0)


    print(f'Потери на валидации {avg_val_loss}')
    print(f'время на валидации {validation_time}')


    # Записываем все статистики для эпохи

    training_stats.append(
    {'epoch': epoch_i + 1,
     'Training loss' : avg_train_loss,
     'Valid f1' : avg_val_f1,
     'Valid loss' : avg_val_loss,
     'Training time' : training_time,
     'Validation time' : validation_time})


    print('')
    print('Обучение готово')
    print(f'Общее время обучения {format_time(time.time() - total_t0)}')


======= Epoch 1 / 10 ========
Batch 40 of 48,0:00:03

Средние потери на train 0.6191277435670296
Время эпохи 0:00:04

Сейчас идет валидациия...
f1_score--->  0.7024248460499573
Потери на валидации 0.6940713638351077
время на валидации 0:00:01

Обучение готово
Общее время обучения 0:00:04

======= Epoch 2 / 10 ========
Batch 40 of 48,0:00:03

Средние потери на train 0.6158431842923164
Время эпохи 0:00:04

Сейчас идет валидациия...
f1_score--->  0.7014479734605717
Потери на валидации 0.698147748197828
время на валидации 0:00:01

Обучение готово
Общее время обучения 0:00:08

======= Epoch 3 / 10 ========
Batch 40 of 48,0:00:03

Средние потери на train 0.6123593536516031
Время эпохи 0:00:04

Сейчас идет валидациия...
f1_score--->  0.7019902735084087
Потери на валидации 0.695172477336157
время на валидации 0:00:01

Обучение готово
Общее время обучения 0:00:13

======= Epoch 4 / 10 ========
Batch 40 of 48,0:00:03

Средние потери на train 0.6133603751659393
Время эпохи 0:00:04

Сейчас идет в

f1 = 0.668 rubert tiny 'cointegrated/rubert-tiny'

f1 = 0.6913812702036449"distilbert-base-multilingual-cased"

f1 = 0.6913812702036449"distilbert-base-multilingual-cased"   10 эпох

f1_score--->  0.6921519089589768                10 эпох

f1_score--->  0.7161098612987867 cointegrated/rubert-tiny2

# выводы
Для улучшения предсказания, необходио подбирать мультиязычную другую модель

In [ ]:
training_stats

[{'epoch': 1,
  'Training loss': 1.1939915182689826,
  'Valid f1': 0.6815396894312968,
  'Valid loss': 1.0456726267224266,
  'Training time': '0:00:06',
  'Validation time': '0:00:00'},
 {'epoch': 2,
  'Training loss': 1.0019345010320346,
  'Valid f1': 0.6705450399153744,
  'Valid loss': 0.9646439807755607,
  'Training time': '0:00:04',
  'Validation time': '0:00:00'},
 {'epoch': 3,
  'Training loss': 0.9118354419867197,
  'Valid f1': 0.6578271189820394,
  'Valid loss': 0.8673279484113058,
  'Training time': '0:00:04',
  'Validation time': '0:00:01'},
 {'epoch': 4,
  'Training loss': 0.8165475701292356,
  'Valid f1': 0.6746288991896828,
  'Valid loss': 0.7909049789110819,
  'Training time': '0:00:04',
  'Validation time': '0:00:00'},
 {'epoch': 5,
  'Training loss': 0.7433799331386884,
  'Valid f1': 0.6830364600305457,
  'Valid loss': 0.7444205142202831,
  'Training time': '0:00:04',
  'Validation time': '0:00:00'},
 {'epoch': 6,
  'Training loss': 0.6971578548351923,
  'Valid f1': 0.6